# Optimizing binning schemes 1: Optimising for stand-alone measurement?

In [1]:
from combine_bins import test_F_1 as F
from combine_bins import test_c_1 as c
from combine_bins import Amp_p

Ap = Amp_p(F, c, 0.3)

In [2]:
from combine_bins.util import get_F_c
from combine_bins import BinnedFitter 
import combine_bins.binnings as binnings

simple_binning = binnings.simple_binning
equal_binning = binnings.approx_equal_binning


In [8]:
import autograd.numpy as np

def Q(amp, binning, x=0):
    Fs, cs = get_F_c(amp, binning)
    bf = BinnedFitter(binning, Fs, cs)
    N = bf.predict_yields(x, 1000)
    dN = bf.predict_dN_dx(x, 1000)
    return np.sqrt( np.sum((dN/np.sqrt(N)) **2) / (sum(N)*amp.sensitivity(x)))

In [18]:
Q_simple = Q(Ap, simple_binning, 0.3)
Q_equal = Q(Ap, equal_binning, 0.3)

Does the factor $Q$ reproduces $1/\sigma_x$?

In [22]:
# Results from the simple studies
mean_cont = 0.015744444629682317
mean_simple = 0.01936067469619681
mean_equal = 0.02063644527315257

In [21]:
print (f'continuous sigma ratio: {mean_cont/mean_simple:1.3f} vs Q: {Q_simple:1.3f}')
print (f'continuous sigma ratio: {mean_cont/mean_equal:1.3f} vs Q: {Q_equal:1.3f}')
print (f'bin vs bin sigma ratio: {mean_equal/mean_simple:1.3f} vs Q: {Q_simple/Q_equal:1.3f}')


continuous sigma ratio: 0.813 vs Q: 0.905
continuous sigma ratio: 0.763 vs Q: 0.881
bin vs bin sigma ratio: 1.066 vs Q: 1.027


So it does not seem like the ratios are produced exactly, but it does look like the **ordering** is correct, and hence $Q$ is tool that can be used in binning-scheme optimisation.

## Making an optimized binning scheme

# Optimising binning scheme 2: choosing optimal binning schemes *to combine*

The final measure to optimise is of course the combined $\sigma_x$. Can we find some proxy variable, like $Q$ is for the single-binning scheme, which avoids spending ages on toy studies?